In [24]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython==8.21.0
!pip install numpy
!pip install sympy


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
49 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [25]:
from manim import *

config.media_embed = True

In [35]:
%%manim -ql -v WARNING DijkstraVisualization

from manim import *
import numpy as np
import heapq
from sympy import symbols, Eq, solve

class DijkstraVisualization(Scene):
    def construct(self):
        # Paleta de Cores
        node_color = BLUE_B
        node_text_color = WHITE
        edge_color = GREY
        path_color = GOLD

        # Definir os nós do grafo e suas posições
        nodes = {
            "A": LEFT * 3 + UP * 2,
            "B": LEFT * 2 + DOWN * 1,
            "C": LEFT * 1 + UP * 1,
            "D": RIGHT * 1 + UP * 1,
            "E": RIGHT * 2 + DOWN * 1,
            "F": RIGHT * 3 + UP * 2
        }

        # Criar círculos e rótulos para os nós
        node_circles = {
            node: Circle(radius=0.6, color=node_color, fill_opacity=0.7).move_to(position)
            for node, position in nodes.items()
        }

        node_labels = {
            node: Text(node, color=node_text_color, font_size=30).move_to(position)
            for node, position in nodes.items()
        }

        # Definir as arestas e pesos
        edges = [
            ("A", "B", 4),
            ("A", "C", 2),
            ("B", "C", 5),
            ("B", "D", 10),
            ("C", "D", 3),
            ("C", "E", 8),
            ("D", "F", 7),
            ("E", "F", 4)
        ]

        # Criar matriz de adjacência
        node_indices = {node: i for i, node in enumerate(nodes.keys())}
        adjacency_matrix = np.zeros((len(nodes), len(nodes)))

        for start, end, weight in edges:
            i, j = node_indices[start], node_indices[end]
            adjacency_matrix[i, j] = weight
            adjacency_matrix[j, i] = weight

        # Função para criar as arestas
        def get_edge(start, end):
            start_pos = nodes[start]
            end_pos = nodes[end]
            return Line(start_pos, end_pos).set_color(edge_color)

        edge_lines = [get_edge(start, end) for start, end, _ in edges]

        edge_labels = {}
        for start, end, weight in edges:
            mid_point = (nodes[start] + nodes[end]) / 2
            label = Text(str(weight), color=WHITE, font_size=24).move_to(mid_point)
            edge_labels[(start, end)] = label

        # Animação de criação do grafo
        self.play(
            *[FadeIn(circle) for circle in node_circles.values()],
            *[Write(label) for label in node_labels.values()],
            *[Create(edge) for edge in edge_lines],
            *[Write(label) for label in edge_labels.values()]
        )

        self.wait(1)

        # Implementação do algoritmo de Dijkstra
        def dijkstra(graph, start_index):
            n = len(graph)
            distances = np.full(n, np.inf)
            distances[start_index] = 0
            previous_nodes = np.full(n, -1)
            visited = set()
            queue = [(0, start_index)]

            while queue:
                (dist, current_node) = heapq.heappop(queue)
                if current_node in visited:
                    continue
                visited.add(current_node)

                for neighbor in range(n):
                    if graph[current_node, neighbor] > 0 and neighbor not in visited:
                        alt = dist + graph[current_node, neighbor]
                        if alt < distances[neighbor]:
                            distances[neighbor] = alt
                            previous_nodes[neighbor] = current_node
                            heapq.heappush(queue, (alt, neighbor))

            return previous_nodes, distances

        # Executar Dijkstra e obter resultados
        start_node = "A"
        end_node = "F"
        start_index = node_indices[start_node]
        end_index = node_indices[end_node]

        previous_nodes, distances = dijkstra(adjacency_matrix, start_index)

        # Determinar o caminho mais curto
        path = []
        current = end_index
        while current != -1:
            path.append(current)
            current = int(previous_nodes[current])
        path = path[::-1]

        # Converter índices de volta para os nomes dos nós
        path_names = [list(nodes.keys())[i] for i in path]

        # Destacar o caminho mais curto
        path_lines = [get_edge(path_names[i], path_names[i + 1]) for i in range(len(path_names) - 1)]
        self.play(*[Create(line.set_color(path_color)) for line in path_lines])

        final_text = Text(f"Caminho mais curto: {' -> '.join(path_names)}", color=WHITE, font_size=24).to_edge(DOWN)
        self.play(Write(final_text))

        # Resolver sistema linear baseado no grafo
        x1, x2, x3, x4, x5, x6, x7, x8 = symbols('x1 x2 x3 x4 x5 x6 x7 x8')

        # Criar equações baseadas nos pesos e fluxos das arestas
        equations = [
            Eq(x1 + x2, 6),  # Total de fluxo entrando no nó A
            Eq(x1 - x3 + x4, 0),  # Conservação de fluxo no nó B
            Eq(x2 + x3 - x5 - x6, 0),  # Conservação de fluxo no nó C
            Eq(x4 + x6 - x7, 0),  # Conservação de fluxo no nó D
            Eq(x5 - x8, 0),  # Conservação de fluxo no nó E
            Eq(x7 + x8, 11)  # Total de fluxo saindo do nó F
        ]

        # Resolver sistema de equações
        solution = solve(equations, (x1, x2, x3, x4, x5, x6, x7, x8))

        # Exibir resultado na animação
        algebra_text = Text(f"Solução dos fluxos: {solution}", color=WHITE, font_size=24).to_edge(UP)
        self.play(Write(algebra_text))

        self.wait(3)


Manim Community v0.18.1

In [34]:
!ls media/videos/content/720p30




ls: cannot access 'media/videos/content/720p30': No such file or directory


In [ ]:
from IPython.display import Video

# Caminho para o vídeo gerado
video_path = 'media/videos/content/480p15/DijkstraVisualization.mp4'

# Exibir o vídeo diretamente no Colab com embed
Video(video_path, embed=True)
